Bankruptcy Prediction Analysis

In [1]:
import pandas as pd
import os
import statsmodels.api as sm
from sklearn.metrics import accuracy_score, confusion_matrix

os.getcwd()

df = pd.read_csv('Data/american_bankruptcy.csv')

In [23]:

df.head(5)


,company_name,status_label,year,ca,cogs,d_and_a,ebitda,inventory,ni,ar,...,mv_to_tl,tr_to_ta,z_score,latest_period,default,wc_to_ta_pct,re_to_ta_pct,ebit_to_ta_pct,mv_to_tl_pct,tr_to_ta_pct
0,C_1,alive,1999,511.267,833.107,18.373,89.031,336.018,35.163,128.348,...,0.928438,1.382369,3.196587,0,0,0.792646,0.747960,0.714051,0.261521,0.724359
1,C_1,alive,2000,485.856,713.811,18.577,64.367,320.590,18.531,115.187,...,1.042794,1.245637,3.109968,0,0,0.828436,0.762627,0.580489,0.289113,0.673915
2,C_1,alive,2001,436.656,526.477,22.496,27.207,286.588,-58.939,77.528,...,0.911564,0.899355,2.226949,0,0,0.732264,0.687120,0.376249,0.257276,0.505961
3,C_1,alive,2002,396.412,496.747,27.172,30.745,259.954,-12.410,66.322,...,0.365979,0.883074,1.709901,0,0,0.595638,0.673458,0.373198,0.103937,0.496657
4,C_1,alive,2003,432.204,523.302,26.680,47.491,247.245,3.504,104.661,...,0.757853,0.919167,2.240159,0,0,0.752599,0.678186,0.434572,0.217381,0.516815


In [4]:
df = df.rename(columns={'X1':'ca','X2':'cogs','X3':'d_and_a','X4':'ebitda','X5':'inventory','X6':'ni','X7':'ar','X8':'mv','X9':'ns','X10':'ta','X11':'tltd','X12':'ebit','X13':'gp','X14':'cl','X15':'re','X16':'tr','X17':'tl','X18':'toe'})


In [5]:
df['wc'] = df['ca'] - df['cl']
df['wc_to_ta'] = df['wc'] / df['ta']
df['re_to_ta'] = df['re'] / df['ta']
df['ebit_to_ta'] = df['ebit'] / df['ta']
df['mv_to_tl'] = df['mv'] / df['tl']
df['tr_to_ta'] = df['tr'] / df['ta']

df['z_score'] = 1.2*df['wc_to_ta'] + 1.4*df['re_to_ta'] + 3.3*df['ebit_to_ta'] + 0.6*df['mv_to_tl'] + 1.0*df['tr_to_ta']

df['latest_period'] = df['year'] == df.groupby('company_name')['year'].transform('max')
df['latest_period'] = df['latest_period'].astype(int)

df['default'] = ((df['latest_period'] == 1) & (df['status_label'] == 'failed')).astype(int)

In [11]:
training_features = ['wc_to_ta','re_to_ta','ebit_to_ta','mv_to_tl','tr_to_ta']

x_train = pd.DataFrame()

for column in training_features:
    x_train[column + '_pct'] = df[column].rank(method='min', pct=True)

y_train = df[['default']]

In [17]:
model = sm.Logit(y_train, x_train)
result = model.fit()


Optimization terminated successfully.
         Current function value: 0.040693
         Iterations 11


In [ ]:
predictions = result.predict(x_train)
predicted_classes = (predictions > 0.5).astype(int)

In [19]:
accuracy = accuracy_score(y_test, test_predicted_classes)
conf_matrix = confusion_matrix(y_test, test_predicted_classes)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

NameError: name 'accuracy_score' is not defined

In [18]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                default   No. Observations:                78682
Model:                          Logit   Df Residuals:                    78677
Method:                           MLE   Df Model:                            4
Date:                Thu, 18 Apr 2024   Pseudo R-squ.:                  0.1024
Time:                        15:12:23   Log-Likelihood:                -3201.8
converged:                       True   LL-Null:                       -3567.2
Covariance Type:            nonrobust   LLR p-value:                7.604e-157
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
wc_to_ta_pct      -0.6404      0.167     -3.823      0.000      -0.969      -0.312
re_to_ta_pct      -2.1719      0.260     -8.369      0.000      -2.681      -1.663
ebit_to_ta_pct    -2.8662      0.267    -10.731      0.000      -3.390      -2.343
mv_to_tl_pct      -6.6377      0.231    -28.728      0.000      -7.091      -6.185
tr_to_ta_pct      -1.6806      0.096    -17.464      0.000      -1.869      -1.492
==================================================================================

Possibly complete quasi-separation: A fraction 0.20 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""